In [1]:
import chess
import numpy as np
import random
import os
import pickle
os.chdir("/Users/laurensuarez/Documents/blake/turnouts/")
file_path = "turnouts.pkl"

import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.losses import mean_squared_error
import numpy
numpy.random.seed(7)

/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

## Load 3000 games

In [3]:
# Save turnouts
import os
import pickle
os.chdir("/Users/laurensuarez/Documents/blake/turnouts/")
turnouts=pickle_load('turnouts.pkl')

winningTensorsList=list(map(lambda x:x[0][2], turnouts['1-0']))

losingTensorsList=list(map(lambda x:x[0][2], turnouts['0-1']))

tieingTensorsList=list(map(lambda x:x[0][2], turnouts['1/2-1/2']))

N=len(winningTensorsList)+len(losingTensorsList)+len(tieingTensorsList)

df=np.ndarray((N, 
                           winningTensorsList[0].shape[0],
                           winningTensorsList[0].shape[1],
                           winningTensorsList[0][0].shape[1]))
for i in range(len(winningTensorsList)):
    df[i]=winningTensorsList[i]

print(Wrewards.shape)
print(Brewards.shape)
print(df.shape)

(3003,)
(3003,)
(3003, 8, 8, 13)


## Pretrain Friendly and Threat AI
### 3000 points

In [38]:
class Agent(object):
    
    model=None
    st_memory_X=[]
    st_memory_y=[]
    lt_memory=[]
    name=""
    def __init__(self,name):
        self.name=name
        self.build_model()

    def forget_st_X(self):
        self.st_memory_X=[]
    
    def forget_st_y(self):
        self.st_memory_y=[]
    
    def remember_st_X(self, point):
        self.st_memory_X.append(point)
        
    def remember_st_y(self, success):
        self.st_memory_y=[success for i in range(len(self.st_memory_X))]
    
    def remember_lt(self, fit=False):
        print("*** Committing to memory ***")
        y=np.array(self.st_memory_y, dtype=np.int)
        X=np.ndarray((len(self.st_memory_X), 8,8,13))
        for i in range(len(self.st_memory_X)):
            X[i]=self.st_memory_X[i]
            
        self.lt_memory.append((X,y))
        #if fit:
        self.fit_data(X,y)
        
        self.forget_st_X()
        self.forget_st_y()

    def build_model(self):
        
        model = Sequential()
        model.add(Dense(1028, activation='sigmoid', input_shape=(8,8,13)))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Flatten())
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss="mean_squared_error", 
                      optimizer=keras.optimizers.SGD(lr=0.01), 
                      metrics=['mae'])
        
        self.model=model
        return self.model
        
    def fit_data(self, Xtrain, ytrain):
        print("*** Learning from memory ***")
        print("AI " + self.name + "\tLearning.")
        assert(self.model!=None), "Model isnt built yet, run `build_model` first."
        self.model.fit(Xtrain, ytrain,
                  batch_size=10)
                  #epochs=epochs,
                  #verbose=1,
                  #validation_data=(Xtest, ytest),
                  #callbacks=[history]
                  #)

    
    def predict(self, point):
        return self.model.predict(np.expand_dims(point,axis=0))


a0=agents[0]
a1=agents[1]

In [52]:
# Save turnouts
import os
import pickle
os.chdir("/Users/laurensuarez/Documents/blake/turnouts/")
turnouts=pickle_load('turnouts.pkl')

winningTensorsList=list(map(lambda x:x[0][2], turnouts['1-0']))
WwinReward=np.array([1 for i in range(len(winningTensorsList))], np.int)
BwinReward=np.array([-1 for i in range(len(winningTensorsList))], np.int)

losingTensorsList=list(map(lambda x:x[0][2], turnouts['0-1']))
WlosingReward=np.array([-1 for i in range(len(losingTensorsList))], np.int)
BlosingReward=np.array([1 for i in range(len(losingTensorsList))], np.int)

tieingTensorsList=list(map(lambda x:x[0][2], turnouts['1/2-1/2']))
WtieingReward=np.array([0 for i in range(len(tieingTensorsList))], np.int)
BtieingReward=np.array([0 for i in range(len(tieingTensorsList))], np.int)

N=len(winningTensorsList)+len(losingTensorsList)+len(tieingTensorsList)
df=np.ndarray((N, 
                           winningTensorsList[0].shape[0],
                           winningTensorsList[0].shape[1],
                           winningTensorsList[0][0].shape[1]))
for i in range(len(winningTensorsList)):
    df[i]=winningTensorsList[i]

Wrewards=np.hstack((WwinReward,WlosingReward,WtieingReward))
Brewards=np.hstack((BwinReward,BlosingReward,BtieingReward))

print(Wrewards.shape)
print(Brewards.shape)
print(df.shape)

X=df
yW=Wrewards
yB=Brewards

a3=agents[2]
a4=agents[3]

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, yW, test_size=0.33, shuffle=True, random_state=42)
a3.model.fit(X, yW, batch_size=10)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, yB, test_size=0.33, shuffle=True, random_state=42)
a4.model.fit(X, yB, batch_size=10)

(3003,)
(3003,)
(3003, 8, 8, 13)
Epoch 1/1
3003/3003 [==============================] - 464s 155ms/step - loss: 0.6667 - mean_absolute_error: 0.6667
Epoch 1/1
3003/3003 [==============================] - 463s 154ms/step - loss: 0.6669 - mean_absolute_error: 0.6662


In [56]:

#Winner Lookup
bool2Winner={True:"White",
             False:"Black"}
#Piece Lookup
piece2int={'.':0,'P':chess.PAWN,'R':chess.ROOK,'N':chess.KNIGHT,'B':chess.BISHOP,'Q':chess.QUEEN,'K':chess.KING,
                             'p':chess.PAWN*-1,'r':chess.ROOK*-1,'n':chess.KNIGHT*-1,'b':chess.BISHOP*-1,'q':chess.QUEEN*-1,'k':chess.KING*-1}

domain=list(piece2int.values())

        
#Convert chess value to one_hot encoding
def create_one_hot(pieceId):
    one_hot=[]
    for i,j in enumerate(domain):
        if(j==pieceId):
            one_hot.append(1)
        else:
            one_hot.append(0)
    return np.array(one_hot)

#Construct tensor from matrix
def create_tensor(M):
    rows,cols=M.shape
    T=np.zeros((rows,cols,len(domain)), dtype=np.int)
    for i in range(rows):
        for j in range(cols):
            T[i,j]=create_one_hot(M[i,j])
    return T

#Convert list of list of string board to matrix
#board2Matrix=(lambda someBoard: np.matrix(list(map(lambda x:list(map(lambda y:y.replace(y,str(self.domain)), x.split(" "))), str(someBoard).split("\n"))), dtype=np.int))
def board2Matrix(someBoard):
    return np.matrix(list(map(lambda x:list(map(lambda y:y.replace(y,str(piece2int[y])), x.split(" "))), str(someBoard).split("\n"))), dtype=np.int)

#Convert matrix to tensor
#board2Tensor=(lambda someBoard: create_tensor(board2Matrix(someBoard), self.domain))
def board2Tensor(someBoard):
    return create_tensor(board2Matrix(someBoard))

"""
a1=agents[0]
a2=agents[1]
turn=1
winCount=0
lossCount=0
tieCount=0
"""
while True:
    
    newBoard = chess.Board()
    sequenceCount=0
    
    while newBoard.is_game_over()==False:
        
        moves=list(newBoard.legal_moves)
        if turn==1:
            # Get best move
            if winCount > 100:
                whitePreds=[]
                for m in moves:
                    newBoard.push(move)
                    dataPoint=board2Tensor(newBoard)
                    whitePreds.append(a3.predict(dataPoint)[0][0])
                    newBoard.pop()
                bestMove=moves[np.argmax(whitePreds)]
                print(whitePreds)
                #print(np.argmax(whitePreds))
                #input("...")
            else:
                bestMove=random.choice(moves)
            
            # Save move
            dataPoint=board2Tensor(newBoard)
            a3.remember_st_X(dataPoint)
            
            # Execute
            newBoard.push(bestMove)
            
            # Alternative AI
            turn=-1
        else:
            if lossCount > 100:
            # Get best move
                blackPreds=[]
                for m in moves:
                    newBoard.push(move)
                    dataPoint=board2Tensor(newBoard)
                    blackPreds.append(a4.predict(dataPoint)[0][0])
                    newBoard.pop()
                print(blackPreds)
                bestMove=moves[np.argmax(blackPreds)]
                #print(np.argmax(blackPreds))
                #input("...")
            else:
                bestMove=random.choice(moves)
            
            # Save move
            dataPoint=board2Tensor(newBoard)
            a4.remember_st_X(dataPoint)
            
            # Execute
            newBoard.push(bestMove)
            
            # Alternate AI
            turn=1
        sequenceCount+=1
    
    winner=newBoard.result()
    print("Winner: " + winner)
    
    if winner == '1-0':
        print("#Moves: " + str(len(a1.st_memory_X)) )
        print(newBoard)
    if winner == '0-1':
        print("#Moves: " + str(len(a2.st_memory_X)) )
        print(newBoard)
    
    if winner == '1-0':
        #a1 won; commit reward to a1 penalty to a2
        a3.remember_st_y(1)
        a3.remember_lt()
        a4.remember_st_y(-1)
        a4.remember_lt()
        winCount+=1
    elif winner == '0-1':
        #a2 won; commit reward to a2 penalty to a1
        a3.remember_st_y(-1)
        a3.remember_lt()
        a4.remember_st_y(1)
        a4.remember_lt()
        lossCount+=1
    elif winner == '1/2-1/2':
        #neither won, commit without any reward; 0.0
        a3.remember_st_y(-1)
        a3.remember_lt()
        a4.remember_st_y(-1)
        a4.remember_lt()
        tieCount+=1
    else:
        print("Error.")
        
    print("*** Status *** ")
    print("Wins: " + str(winCount))
    print("Losses: " + str(lossCount))
    print("Ties: " + str(tieCount))


Winner: 1/2-1/2
Committing to memory
Epoch 1/1
258/258 [==============================] - 27s 106ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
257/257 [==============================] - 28s 108ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 13
Losses: 11
Ties: 259
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
201/201 [==============================] - 22s 107ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
202/202 [==============================] - 25s 122ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 13
Losses: 11
Ties: 260
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
133/133 [==============================] - 16s 119ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
133/133 [==============================] - 14s 108ms/step - loss: 1.0000 - mean_abso

AI 2	Learning.
Committing to memory
Epoch 1/1
155/155 [==============================] - 15s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 14
Losses: 11
Ties: 280
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
144/144 [==============================] - 15s 103ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
144/144 [==============================] - 15s 102ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 14
Losses: 11
Ties: 281
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
183/183 [==============================] - 18s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
182/182 [==============================] - 18s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 14
Losses: 11
Ties: 282
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
269/269 [==============

Winner: 1/2-1/2
Committing to memory
Epoch 1/1
186/186 [==============================] - 18s 99ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
187/187 [==============================] - 19s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 18
Losses: 15
Ties: 315
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
205/205 [==============================] - 21s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
205/205 [==============================] - 21s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 18
Losses: 15
Ties: 316
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
203/203 [==============================] - 21s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
203/203 [==============================] - 20s 101ms/step - loss: 1.0000 - mean_absol

135/135 [==============================] - 14s 103ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 20
Losses: 18
Ties: 353
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
128/128 [==============================] - 13s 99ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
128/128 [==============================] - 13s 99ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 20
Losses: 18
Ties: 354
Winner: 1-0
#Moves: 2
k b . . . . . .
. . . . . . . .
. . Q . . . . .
p . . . . K . p
P . . . . . n P
. . p . . . P .
b . . . . B . R
. . . . . . N .
Committing to memory
Epoch 1/1
77/77 [==============================] - 8s 104ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
76/76 [==============================] - 8s 105ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 21
Losses: 18


70/70 [==============================] - 7s 107ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 24
Losses: 21
Ties: 389
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
125/125 [==============================] - 12s 99ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
125/125 [==============================] - 12s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 24
Losses: 21
Ties: 390
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
212/212 [==============================] - 21s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
212/212 [==============================] - 21s 100ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 24
Losses: 21
Ties: 391
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
149/149 [==============================] - 15s 100ms/step - loss: 1.0000 

Winner: 1/2-1/2
Committing to memory
Epoch 1/1
200/200 [==============================] - 20s 102ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
200/200 [==============================] - 20s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 28
Losses: 21
Ties: 407
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
235/235 [==============================] - 24s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
236/236 [==============================] - 24s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 28
Losses: 21
Ties: 408
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
231/231 [==============================] - 23s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
231/231 [==============================] - 23s 101ms/step - loss: 1.0000 - mean_abso

49/49 [==============================] - 5s 105ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
50/50 [==============================] - 5s 105ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 29
Losses: 22
Ties: 427
Winner: 0-1
#Moves: 2
B . N . . . . .
. . r . . p k .
. . . . . . . .
p . . . p P . R
P . . B n . p .
. r . n . b . .
. . P . . . . .
Q . K . . . . .
Committing to memory
Epoch 1/1
58/58 [==============================] - 6s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
58/58 [==============================] - 6s 103ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 29
Losses: 23
Ties: 427
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
141/141 [==============================] - 14s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
140/140 [

Winner: 1/2-1/2
Committing to memory
Epoch 1/1
324/324 [==============================] - 33s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
324/324 [==============================] - 33s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 29
Losses: 25
Ties: 446
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
111/111 [==============================] - 11s 103ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
112/112 [==============================] - 11s 102ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 3	Learning.
*** Status *** 
Wins: 29
Losses: 25
Ties: 447
Winner: 1/2-1/2
Committing to memory
Epoch 1/1
265/265 [==============================] - 27s 101ms/step - loss: 1.0000 - mean_absolute_error: 1.0000
AI 2	Learning.
Committing to memory
Epoch 1/1
265/265 [==============================] - 27s 101ms/step - loss: 1.0000 - mean_abso

KeyboardInterrupt: 

### Pretrained Friendly vs. Pretrained Threat


In [ ]:
newBoard=chess.Board()
while newBoard.is_game_over()==False:
        
    moves=list(newBoard.legal_moves)
    if turn==1:
        # Get best move
        whitePreds=[]
        for m in moves:
            newBoard.push(move)
            dataPoint=board2Tensor(newBoard)
            whitePreds.append(a3.predict(dataPoint)[0][0])
            newBoard.pop()
        bestMove=moves[np.argmax(whitePreds)]
        print(whitePreds)
        #print(np.argmax(whitePreds))
        input("...")

        # Save move
        dataPoint=board2Tensor(newBoard)
        a3.remember_st_X(dataPoint)

        # Execute
        newBoard.push(bestMove)

        # Alternative AI
        turn=-1
    else:
        # Get best move
        blackPreds=[]
        for m in moves:
            newBoard.push(move)
            dataPoint=board2Tensor(newBoard)
            blackPreds.append(a4.predict(dataPoint)[0][0])
            newBoard.pop()
        print(blackPreds)
        #print(np.argmax(blackPreds))
        input("...")

        # Save move
        dataPoint=board2Tensor(newBoard)
        a4.remember_st_X(dataPoint)

        # Execute
        newBoard.push(bestMove)

        # Alternate AI
        turn=1
    sequenceCount+=1

winner=newBoard.result()
print(winner)

[2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10, 2.0758092e-10]


### Save model

In [ ]:
from keras.models import load_model

a1.model.save('a1_model.h5')  # creates a HDF5 file 'my_model.h5'
a1.model = load_model('a1_model.h5')
a2.model.save('a2_model.h5')  # creates a HDF5 file 'my_model.h5'
a2.model = load_model('a2_model.h5')
#del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
